<a href="https://colab.research.google.com/github/venkatatharu/wings_Personal-Finance-Chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch gradio accelerate bitsandbytes

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import gradio as gr

# Configure model loading
model_name = "ibm-granite/granite-3.2-2b-instruct"

# Quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

print("Model loaded successfully!")

def chat_response(message, history):
    # System prompt
    system = "You are ClauseWise, a personal finance advisor. Give concise advice on savings, taxes, and investments."

    # Build conversation
    prompt = f"<|system|>\n{system}\n<|user|>\n{message}\n<|assistant|>\n"

    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract assistant response
    response = full_response.split("<|assistant|>")[-1].strip()

    return response

# Create interface
demo = gr.ChatInterface(
    fn=chat_response,
    title="🏦 ClauseWise - Personal Finance Chatbot",
    description="Get advice on savings, taxes, and investments",
    examples=[
        "How much emergency fund do I need?",
        "Best way to start investing $1000?",
        "How to reduce tax burden?",
        "Pay debt first or invest?"
    ],
)

# Launch
demo.launch(debug=True, share=True)

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://83cc1ef79118e8007f.gradio.live
